In [25]:
import numpy as np

from src.models.train_emos import train_emos, train_and_test_emos
from pit import make_cpit_diagram_emos, make_cpit_hist_emos 
from brier_score import brier_skill_plot, brier_plot
from src.models.get_data import get_tensors, get_normalized_tensor
from src.models.emos import EMOS

In [2]:
neighbourhood_size = 11
parameter_names = ['wind_speed', 'press', 'kinetic', 'humid', 'geopot']
ignore = ['229', '285', '323']
train_folds = [1, 2]
train_data = get_normalized_tensor(neighbourhood_size, parameter_names, train_folds, ignore)

X_train = train_data['X']
y_train = train_data['y']
variances_train = train_data['variances']
mean_train = train_data['mean']
std_train = train_data['std']


print(X_train.shape)

(15733, 5)


In [3]:
test_fold = 3

X_test, y_test, variances_test = get_tensors(neighbourhood_size, parameter_names, test_fold, ignore)
X_test = (X_test - mean_train) / std_train

print(X_test.shape)

(7780, 5)


In [4]:
setup = {}

setup["num_features"] = len(parameter_names)
setup["feature_mean"] = mean_train
setup["feature_std"] = std_train
setup["features"] = parameter_names
setup["neighbourhood_size"] = neighbourhood_size

Possible settings for EMOS class:

loss:

    loss_CRPS_sample
    loss_log_likelihood
    loss_Brier_score 
    loss_twCRPS_sample

in case sample is used loss: sample

chaining_functions

    chain_function_indicator 
        includes:
        
         threshold
    chain_function_normal_cdf
        includes: 
        
        chain_function_mean
         chain_function_std

optimizer:

    Adam
    SGD

learning_rate:
    -Positive real valued number

forecast_distribution:

    distr_trunc_normal
    distr_log_normal
    distr_gev(2/3)

    distr_mixture (which contains two of the above distributions)
    distr_mixture_linear

they must contain distribution_1/distribution_2

    


In [5]:
models_crps = {}
setup1 = setup

setup1["loss"] = "loss_CRPS_sample"
setup1["samples"] = 100
setup1["optimizer"] = "Adam"
setup1["learning_rate"] = 0.01
setup1["forecast_distribution"] = "distr_trunc_normal"

trunc_normal_crps = EMOS(setup1)

setup1["forecast_distribution"] = "distr_log_normal"

log_normal_crps = EMOS(setup1)

setup1["forecast_distribution"] = "distr_gev"
setup1["samples"] = 300

gev_crps = EMOS(setup1)

setup1["forecast_distribution"] = "distr_gev2"

gev2_crps = EMOS(setup1)

setup1["forecast_distribution"] = "distr_gev3"

gev3_crps = EMOS(setup1)


models_crps["trunc_normal_crps"] = trunc_normal_crps
models_crps["log_normal_crps"] = log_normal_crps
models_crps["gev_crps"] = gev_crps
#models_crps["gev2_crps"] = gev2_crps
#models_crps["gev3_crps"] = gev3_crps
print(len(models_crps))

Using default parameters for truncated normal distribution
Using default parameters for Log Normal distribution
Using default parameters for Generalized Extreme Value distribution
Using default parameters for Generalized Extreme Value distribution 2
Using default parameters for Generalized Extreme Value distribution 3
3


In [7]:
epochs = 200

for model in models_crps:
    models_crps[model].fit(X_train, y_train, variances_train, epochs, printing=False)
    print("Model: ", models_crps[model])

Model:  EMOS Model Information:
Loss function: loss_CRPS_sample (Samples: 100)
Forecast distribution: distr_trunc_normal
Parameters:
  a_tn: [0.3458853]
  b_tn: [ 0.97427243 -0.00242053 -0.15589155  0.04777402  0.00460129]
  c_tn: [1.5089761]
  d_tn: [1.2106253]
Features: wind_speed, press, kinetic, humid, geopot
Number of features: 5
Neighbourhood size: 11

Optimizer: Adam
Learning rate: 0.009999999776482582

Model:  EMOS Model Information:
Loss function: loss_CRPS_sample (Samples: 100)
Forecast distribution: distr_log_normal
Parameters:
  a_ln: [0.84415966]
  b_ln: [ 0.13473257 -0.07395755 -0.03527635 -0.02097038  0.06701108]
  c_ln: [0.1215006]
  d_ln: [-0.00185268]
Features: wind_speed, press, kinetic, humid, geopot
Number of features: 5
Neighbourhood size: 11

Optimizer: Adam
Learning rate: 0.009999999776482582

Model:  EMOS Model Information:
Loss function: loss_CRPS_sample (Samples: 300)
Forecast distribution: distr_gev
Parameters:
  a_gev: [0.45766783]
  b_gev: [ 0.850559   -0.

In [8]:
setup1["forecast_distribution"] = "distr_mixture"
setup1["distribution_1"] = "distr_trunc_normal"
setup1["distribution_2"] = "distr_log_normal"

mixture_trunc_log_crps = EMOS(setup1)
mixture_trunc_log_crps.set_parameters(models_crps["trunc_normal_crps"].get_parameters())
mixture_trunc_log_crps.set_parameters(models_crps["log_normal_crps"].get_parameters())

setup1["distribution_2"] = "distr_gev"
mixture_trunc_gev_crps = EMOS(setup1)

mixture_trunc_gev_crps.set_parameters(models_crps["trunc_normal_crps"].get_parameters())
mixture_trunc_gev_crps.set_parameters(models_crps["gev_crps"].get_parameters())

models_crps["mixture_trunc_log_crps"] = mixture_trunc_log_crps
models_crps["mixture_trunc_gev_crps"] = mixture_trunc_gev_crps




Using default parameters for truncated normal distribution
Using default parameters for Log Normal distribution
Parameter a_tn set to [0.3458853]
Parameter b_tn set to [ 0.97427243 -0.00242053 -0.15589155  0.04777402  0.00460129]
Parameter c_tn set to [1.5089761]
Parameter d_tn set to [1.2106253]
Parameter a_ln set to [0.84415966]
Parameter b_ln set to [ 0.13473257 -0.07395755 -0.03527635 -0.02097038  0.06701108]
Parameter c_ln set to [0.1215006]
Parameter d_ln set to [-0.00185268]
Using default parameters for truncated normal distribution
Using default parameters for Generalized Extreme Value distribution
Parameter a_tn set to [0.3458853]
Parameter b_tn set to [ 0.97427243 -0.00242053 -0.15589155  0.04777402  0.00460129]
Parameter c_tn set to [1.5089761]
Parameter d_tn set to [1.2106253]
Parameter a_gev set to [0.45766783]
Parameter b_gev set to [ 0.850559   -0.01507859 -0.05703494 -0.00493977 -0.00441579]
Parameter c_gev set to [0.47593138]
Parameter d_gev set to [ 0.20516989  0.0010

In [9]:
models_crps["mixture_trunc_gev_crps"].fit(X_train, y_train, variances_train, epochs, printing=False)
print("Model: ", models_crps["mixture_trunc_gev_crps"])

models_crps["mixture_trunc_log_crps"].fit(X_train, y_train, variances_train, epochs, printing=False)
print("Model: ", models_crps["mixture_trunc_log_crps"])

Model:  EMOS Model Information:
Loss function: loss_CRPS_sample (Samples: 300)
Forecast distribution: distr_mixture
Distribution 1: distr_trunc_normal
Distribution 2: distr_gev
Mixture weight: [0.7685883]Parameters:
  a_tn: [0.20413199]
  b_tn: [ 0.96398425  0.03421698 -0.21862257  0.00612877  0.10558406]
  c_tn: [1.9902797]
  d_tn: [1.3468198]
  a_gev: [0.29565957]
  b_gev: [ 0.83632994  0.00238316 -0.10161182 -0.04053925  0.07101916]
  c_gev: [0.72616935]
  d_gev: [ 0.40816647 -0.5372744   0.33014664  0.41954613 -0.6925272 ]
  e_gev: [-0.09786802]
  weight: <tf.Variable 'weight:0' shape=(1,) dtype=float32, numpy=array([0.7685883], dtype=float32)>
Features: wind_speed, press, kinetic, humid, geopot
Number of features: 5
Neighbourhood size: 11

Optimizer: Adam
Learning rate: 0.009999999776482582

Model:  EMOS Model Information:
Loss function: loss_CRPS_sample (Samples: 300)
Forecast distribution: distr_mixture
Distribution 1: distr_trunc_normal
Distribution 2: distr_log_normal
Mixture 

In [22]:
##make a plot for the brier scores


EMOS Model Information:
Loss function: loss_CRPS_sample (Samples: 300)
Forecast distribution: distr_mixture
Distribution 1: distr_trunc_normal
Distribution 2: distr_log_normal
Mixture weight: [0.7855019]Parameters:
  a_tn: [-0.33935237]
  b_tn: [ 1.1139809  -0.2288488  -0.2753035  -0.26212028 -0.35431445]
  c_tn: [1.572163]
  d_tn: [1.8492936]
  a_ln: [0.7917785]
  b_ln: [0.10573271 0.13687696 0.04236956 0.20681936 0.3007708 ]
  c_ln: [0.42939496]
  d_ln: [-0.0408224]
  weight: <tf.Variable 'weight:0' shape=(1,) dtype=float32, numpy=array([0.7855019], dtype=float32)>
Features: wind_speed, press, kinetic, humid, geopot
Number of features: 5
Neighbourhood size: 11

Optimizer: Adam
Learning rate: 0.009999999776482582



In [26]:
threshold = 0
make_cpit_diagram_emos(models_crps, X_test, y_test, variances_test, "CPIT_diagram_CRPS", threshold)

TypeError: 'TruncatedNormal' object is not callable

In [15]:
base_model = models_crps.pop("trunc_normal_crps")
values = np.linspace(0, 20, 200)
brier_skill_plot(base_model, models_crps, X_test, y_test, variances_test, values)

KeyError: 'trunc_normal_crps'